In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
from tqdm.auto import tqdm, trange
import xgboost as xgb
import joblib
from sklearn.model_selection import cross_validate
import zipfile

from src.model import tscv, ClippedOutputRegressor
from src.data import get_feature_cols, df_to_X_y, drop_non_features, add_lagged_features


%run constants.py

baseline_reg = joblib.load(os.path.join(MODELS_DIR, 'xgb-baseline.model'))

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn]:
    print("  %s: %s" %(module.__name__, module.__version__))

Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.1
  numpy: 1.19.1
  seaborn: 0.10.1
  sklearn: 0.23.2


For linear models we have to standardize the features and one-hot encode categorical features.

Let's start with one-hot encoding. All features were created with a prefix `f__` and, when they are categorical features `f__cat__`, so this makes it easy for us to build the encoder.

In [45]:
def make_encoder_step(train_set, test_set):
    features = get_feature_cols(train_set)

    categories = []
    indexes = []

    for i, feature in enumerate(features):
        if feature.startswith('f__cat__'):
            cats = np.union1d(train_set[feature], test_set[feature]).tolist()
            categories.append(cats)
            indexes.append(i)

    return ColumnTransformer(
        [("onehot", OneHotEncoder(categories=categories), indexes)],
        remainder='passthrough')

In [13]:
train_set_008 = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set-features-008.parquet'))
test_set_008 = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'test-set-features-008.parquet'))

In [14]:
train_set_008.describe()

,item_id,shop_id,date_block_num,item_cnt_month,f__cat__item_id,f__cat__shop_id,f__cat__date_block_num,month_id,year_id,f__cat__month_id,...,f__item_cnt_15,f__item_cnt_16,f__item_cnt_18,f__item_cnt_20,f__item_cnt_24,f__item_cnt_26,f__item_cnt_28,f__item_cnt_30,f__item_cnt_31,f__item_cnt_32
count,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4284000.0,4.284000e+06,...,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06,4.284000e+06
mean,1.101940e+04,3.164286e+01,2.350000e+01,2.267862e-01,1.101940e+04,3.164286e+01,2.350000e+01,5.500000e+00,1.5,5.500000e+00,...,1.662414e-01,1.559412e-01,1.355292e-01,1.154360e-01,6.934267e-02,4.976867e-02,3.522526e-02,2.219374e-02,1.736088e-02,9.984827e-03
std,6.252631e+03,1.756189e+01,5.766282e+00,1.105050e+00,6.252631e+03,1.756189e+01,5.766282e+00,3.041382e+00,0.5,3.041382e+00,...,3.117029e+00,3.060134e+00,2.915083e+00,2.754693e+00,2.054427e+00,1.540952e+00,1.149730e+00,5.738286e-01,5.325478e-01,3.434243e-01
min,3.000000e+01,2.000000e+00,1.400000e+01,0.000000e+00,3.000000e+01,2.000000e+00,1.400000e+01,0.000000e+00,1.0,0.000000e+00,...,-4.000000e+00,-4.000000e+00,-4.000000e+00,-4.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00,-2.000000e+00
25%,5.381500e+03,1.600000e+01,1.875000e+01,0.000000e+00,5.381500e+03,1.600000e+01,1.875000e+01,3.000000e+00,1.0,3.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.120300e+04,3.450000e+01,2.350000e+01,0.000000e+00,1.120300e+04,3.450000e+01,2.350000e+01,5.500000e+00,1.5,5.500000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.607150e+04,4.700000e+01,2.825000e+01,0.000000e+00,1.607150e+04,4.700000e+01,2.825000e+01,8.000000e+00,2.0,8.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,2.216700e+04,5.900000e+01,3.300000e+01,1.100000e+01,2.0,1.100000e+01,...,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,9.780000e+02,8.200000e+02,7.660000e+02,2.590000e+02,2.590000e+02,1.690000e+02


In [46]:
trans = make_encoder_step(train_set_008, test_set_008)
trans

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot',
                                 OneHotEncoder(categories=[[30, 31, 32, 33, 38,
                                                            42, 45, 51, 53, 57,
                                                            59, 62, 64, 67, 69,
                                                            70, 72, 76, 80, 83,
                                                            88, 91, 93, 109,
                                                            138, 140, 149, 153,
                                                            154, 166, ...],
                                                           [2, 3, 4, 5, 6, 7,
                                                            10, 12, 14, 15, 16,
                                                            18, 19, 21, 22, 24,
                                                            25, 26, 28, 31, 34,
                                           

In [34]:
Xtrain, ytrain = df_to_X_y(train_set_008)

In [47]:
trans.fit_transform(Xtrain, ytrain)

<4284000x5288 sparse matrix of type '<class 'numpy.float64'>'
	with 117215571 stored elements in Compressed Sparse Row format>